In [1]:
from functions import *

import pandas as pd
import numpy as np
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend

In [2]:
## Constants

constants_eip1559 = {
    "BASEFEE_MAX_CHANGE_DENOMINATOR": 8,
    "TARGET_SIZE": 12500000,
    "MAX_BLOCK_SIZE": 25000000,
    "INITIAL_BASEFEE": 1 * (10 ** 9),
}

constants_legacy = {
    "MAX_BLOCK_SIZE": 12500000,
}

levers = {
    "mempool_length": 100
}

In [3]:
## Classes

class eip1559_transaction:
    def __init__(self,total_gas,fee_cap,premium):
        
        # Inherent txn propoerties 
        self.total_gas_used = total_gas
        self.transaction_hash = secrets.token_bytes(8)
        
        # User adjusted txn properties
        self.fee_cap = fee_cap
        self.premium = premium
        
class legacy_transaction:
    def __init__(self,total_gas,fee):
        
        # Inherent txn properties
        self.total_gas_used = total_gas
        self.transaction_hash = secrets.token_bytes(8)
        
        # User adjusted txn properties 
        self.fee = fee
        
class block:
    def __init__(self,size,transactions):
        self.size = size
        self.transactions = transactions 

In [4]:
## Functions 

def demand_functions_eip1559(params, substep, state_history, previous_state, policy_input):
    
    number_of_transactions_in_mempool = levers['mempool_length']
    demand_dict = {}
    
    a = random.randint(1,3)
    b = random.randint(3,7)
    
    for i in range(number_of_transactions_in_mempool):
        tx = eip1559_transaction(
            total_gas = random.randint(20000,30000),
            premium = a*(10**9),
            fee_cap = b *(10**9)
        )
        demand_dict[tx.transaction_hash] = tx
    
    return ("demand", demand_dict)

def demand_functions_legacy(params, substep, state_history, previous_state, policy_input):
    
    number_of_transactions_in_mempool = levers['mempool_length']
    demand_dict = {}
    
    #a = random.randint(1,3)
    b = random.randint(3,7)
    
    for i in range(number_of_transactions_in_mempool):
        tx = legacy_transaction(
            total_gas = random.randint(20000,30000),
            #premium = a*(10**9),
            fee = b * (10 ** 9)
        )
        demand_dict[tx.transaction_hash] = tx
    
    return ("demand", demand_dict)


def transaction_selection_eip1559(params, substep, state_history, previous_state):
    
    demand_dict = previous_state["demand"]
    basefee = previous_state["basefee"]
    
    size = constants_eip1559["MAX_BLOCK_SIZE"]
    final_fee_transactions = {}
    
    for i in demand_dict.keys():
        if basefee > demand_dict[i].fee_cap:
            continue
        else:
            final_fee_transactions[i] = min(demand_dict[i].premium - basefee, demand_dict[i].fee_cap)
            
    final_fee_transactions = dict(sorted(final_fee_transactions.items(), key=lambda x: x[1], reverse=True))
    
    included_transactions = []
    total_size_used = 0
    
    for i in final_fee_transactions.keys():
        
        if total_size_used < size:
            included_transactions += [demand_dict[i]]
            
        total_size_used += demand_dict[i].total_gas_used
        
    return { "block": block(size = total_size_used, transactions = included_transactions) }
        
def transaction_selection_legacy(params, substep, state_history, previous_state):
    
    demand_dict = previous_state["demand"]
    
    size = constants_legacy["MAX_BLOCK_SIZE"]
    final_fee_transactions = {}
    
    for i in demand_dict.keys():
            final_fee_transactions[i] =  demand_dict[i].fee     
            
    final_fee_transactions = dict(sorted(final_fee_transactions.items(), key=lambda x: x[1], reverse=True))
    
    included_transactions = []
    total_size_used = 0
    
    for i in final_fee_transactions.keys():
        
        if total_size_used < size:
            included_transactions += [demand_dict[i]]
            
        total_size_used += demand_dict[i].total_gas_used
        
    return { "block": block(size = total_size_used, transactions = included_transactions) }
        
def update_basefee(params, substep, state_history, previous_state, policy_input):
    
    gas_used = sum([i.total_gas_used for i in policy_input["block"].transactions])
    basefee = previous_state["basefee"]
    
    basefee = basefee + basefee * (gas_used - constants_eip1559["TARGET_SIZE"]) // constants_eip1559["TARGET_SIZE"] // constants_eip1559["BASEFEE_MAX_CHANGE_DENOMINATOR"]
    
    return ("basefee", basefee)


def record_latest_block(params, substep, state_history, previous_state, policy_input):
    
    block = policy_input["block"]
    
    return ("latest_block", block)

## BASELINE MODEL

### Legacy

In [5]:
psub = [{
    "policies": {},
    "variables": {
        "demand": demand_functions_legacy # step 1
    }
}, {
    "policies": {
        "action": transaction_selection_legacy # step 2
    },
    "variables": {
        "latest_block": record_latest_block # step 3
    }
}]

initial_state = {
    "demand": {},
    "latest_block": block(size = 0, transactions=[])
}

blocks = 30

model = Model(
    initial_state=initial_state,
    state_update_blocks=psub,
)
simulation = Simulation(model=model, timesteps=blocks, runs=1)
experiment = Experiment([simulation])
experiment.engine = Engine(deepcopy=False, drop_substeps=True)
result = experiment.run()
df1 = pd.DataFrame(result)

### EIP1559

In [6]:
psub = [{
    "policies": {},
    "variables": {
        "demand": demand_functions_eip1559 # step 1
    }
}, {
    "policies": {
        "action": transaction_selection_eip1559 # step 2
    },
    "variables": {
        "basefee": update_basefee, # step 3
        "latest_block": record_latest_block 
    }
}]

initial_state = {
    "demand": {},
    "latest_block": block(size = 0, transactions=[]),
    "basefee": 10 * (10 ** 9)
}

blocks = 30

model = Model(
    initial_state=initial_state,
    state_update_blocks=psub,
)
simulation = Simulation(model=model, timesteps=blocks, runs=1)
experiment = Experiment([simulation])
experiment.engine = Engine(deepcopy=False, drop_substeps=True)
result = experiment.run()
df2 = pd.DataFrame(result)

### Plots

In [8]:
df2

,demand,latest_block,basefee,simulation,subset,run,substep,timestep
0,{},<__main__.block object at 0x0000028E8785E2E0>,10000000000,0,0,1,0,0
1,{b'\x00\xa3\xcc\x1a\xcb\xcb%\xb1': <__main__.e...,<__main__.block object at 0x0000028E877F4E50>,8750000000,0,0,1,2,1
2,{b'<\xee\xe9\xe7\x15RA\x92': <__main__.eip1559...,<__main__.block object at 0x0000028E8780F7F0>,7656250000,0,0,1,2,2
3,{b'\xa0\x8c\x15 M\xdcd\xb5': <__main__.eip1559...,<__main__.block object at 0x0000028E98C60190>,6699218750,0,0,1,2,3
4,{b'M\x0c\\\xb5s\x0e\xbf\xad': <__main__.eip155...,<__main__.block object at 0x0000028E98C77AF0>,5861816406,0,0,1,2,4
5,{b'h![\xc10\xb9?\x07': <__main__.eip1559_trans...,<__main__.block object at 0x0000028E98C9B490>,5129089355,0,0,1,2,5
6,"{b'\xe2\x93-\xed\xd4""R\xca': <__main__.eip1559...",<__main__.block object at 0x0000028E98CF7DF0>,4487953185,0,0,1,2,6
7,{b'\xa1K\xb8\x94\xad\x1a\xde\x16': <__main__.e...,<__main__.block object at 0x0000028E98D17790>,4038260320,0,0,1,2,7
8,{b'\x96\xd5E\xeb]m\xb4\xea': <__main__.eip1559...,<__main__.block object at 0x0000028E98CB2130>,3633514735,0,0,1,2,8
9,{b'\xbc\x82\x15TP\xef`\xea': <__main__.eip1559...,<__main__.block object at 0x0000028E98CC7A90>,3269481250,0,0,1,2,9
